# Xata

> [Xata](https://xata.io) 是一个无服务器数据平台，基于 PostgreSQL。它提供了一个与数据库交互的 Python SDK，以及一个用于管理数据的 UI。
> Xata 具有原生的向量类型，可以添加到任何表中，并支持相似性搜索。LangChain 将向量直接插入 Xata，并查询给定向量的最近邻居，以便您可以使用所有 LangChain Embeddings 与 Xata 的集成。

本指南将指导您如何使用 Xata 作为 VectorStore。

## 设置

### 创建用于用作向量存储的数据库

在 [Xata UI](https://app.xata.io) 中创建一个新数据库。您可以随意命名，在此记事本中，我们将使用 `langchain`。
创建一个表，同样，您可以随意命名，但我们将使用 `vectors`。通过 UI 添加以下列：

* `content` 类型为“Text”。这用于存储 `Document.pageContent` 的值。
* `embedding` 类型为“Vector”。使用您计划使用的模型所使用的维度。在此笔记本中，我们使用 OpenAI 嵌入，其维度为 1536。
* `source` 类型为“Text”。此列在此示例中用作元数据列。
* 任何您希望用作元数据的其他列。它们是从 `Document.metadata` 对象填充的。例如，如果 `Document.metadata` 对象中有一个 `title` 属性，您可以创建一个 `title` 列，它将被填充。

让我们先安装我们的依赖：

In [ ]:
%pip install --upgrade --quiet  xata langchain-openai langchain-community tiktoken langchain

我们就把 OpenAI 密钥加载到环境中。如果你还没有密钥，可以创建一个 OpenAI 账户，并在[这个页面](https://platform.openai.com/account/api-keys)上创建密钥。

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

同样，我们需要获取 Xata 的环境变量。您可以通过访问您的[账户设置](https://app.xata.io/settings)来创建新的 API 密钥。要查找数据库 URL，请转到您已创建的数据库的设置页面。数据库 URL 应如下所示：`https://demo-uni3q8.eu-west-1.xata.sh/db/langchain`。

In [ ]:
api_key = getpass.getpass("Xata API key: ")
db_url = input("Xata database URL (copy it from your DB settings):")

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.xata import XataVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

### 创建 Xata 向量存储
让我们导入测试数据集：

In [ ]:
loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

现在创建实际的向量存储，由 Xata 表支持。

In [ ]:
vector_store = XataVectorStore.from_documents(
    docs, embeddings, api_key=api_key, db_url=db_url, table_name="vectors"
)

运行上述命令后，如果您转到 Xata UI，您会看到加载的文档及其嵌入。
要使用已包含向量内容的现有 Xata 表，请初始化 XataVectorStore 构造函数：

In [ ]:
vector_store = XataVectorStore(
    api_key=api_key, db_url=db_url, embedding=embeddings, table_name="vectors"
)

### 相似性搜索

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vector_store.similarity_search(query)
print(found_docs)

### 相似性搜索（带分数 - 向量距离）

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
result = vector_store.similarity_search_with_score(query)
for doc, score in result:
    print(f"document={doc}, score={score}")